합성곱 신경망(CNN)을 구현한 예제 코드.
이 예제에서는 MNIST 데이터셋(손글씨 숫자 이미지)을 사용하여 CNN 모델을 학습하는 과정을 보여줍니다.

# 준비단계

## 라이브러리

In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

## 데이터 준비 및 전처리

In [2]:
# MNIST 데이터셋 로드:

# mnist.load_data() 함수는 MNIST 데이터셋을 다운로드하고, 데이터를 (X_train, y_train), (X_test, y_test)의 형태로 반환합니다.
# 이 데이터셋은 60,000개의 훈련 데이터와 10,000개의 테스트 데이터로 구성되어 있습니다.

# 변수 설명:
# X_train: 훈련 데이터의 이미지들로, 각 이미지가 28x28 크기의 회색조 이미지입니다.
# y_train: 훈련 데이터의 라벨들로, 각 이미지가 나타내는 숫자 클래스(0-9)를 포함합니다.
# X_test: 테스트 데이터의 이미지들로, 새로운 데이터에 대해 모델의 성능을 평가할 때 사용합니다.
# y_test: 테스트 데이터의 라벨들로, X_test 이미지의 실제 숫자 클래스(0-9)를 나타냅니다.
# 데이터 구조:
# X_train과 X_test는 각각 (60000, 28, 28) 및 (10000, 28, 28)의 형상을 가지며, 각각 6만 개와 1만 개의 28x28 이미지 배열로 이루어져 있습니다.
# y_train과 y_test는 각각 (60000,) 및 (10000,)의 형상을 가지며, 각 이미지가 나타내는 숫자 라벨을 포함합니다.

In [3]:
# MNIST 데이터셋 로드 (훈련 데이터와 테스트 데이터 분리)
(X_train, y_train), (X_test, y_test) = mnist.load_data()


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [4]:
# 각 이미지를 28x28x1 형태로 변환하여 CNN이 요구하는 4차원 입력 형식에 맞춥니다.

# reshape:
# X_train.shape[0]과 X_test.shape[0]는 각각 훈련 데이터와 테스트 데이터의 이미지 수입니다.
# reshape 함수를 사용하여 (이미지 수, 28, 28, 1) 형식으로 4차원 텐서로 변환합니다.
# 마지막 차원 1은 채널을 의미하며, 1은 흑백(회색조) 이미지를 나타냅니다. 컬러 이미지의 경우 이 값은 3이 되며, RGB 채널을 나타냅니다.

# astype('float32'):
# 데이터를 float32 형식으로 변환하여 연산의 일관성을 유지합니다.
# 원래 MNIST 데이터셋은 정수형(uint8)으로 되어 있으며, 0~255의 값으로 각 픽셀 밝기를 나타냅니다.

# 이후 작업을 위한 준비:
# CNN 모델에 데이터를 입력할 때, 입력 차원을 (배치 크기, 높이, 너비, 채널 수)로 맞춰야 합니다.
# 이 변환을 통해 데이터는 CNN에서 요구하는 형식에 맞춰져, 이후 모델 학습 시 올바르게 처리될 수 있습니다.
# 전처리 요약
# 이 코드는 MNIST 데이터를 CNN 모델에 적합한 형식(28x28x1)으로 변환하고, 연산 효율성을 위해 float32 형식으로 변환합니다.

In [5]:
# 데이터 전처리 (28x28 크기의 이미지를 28x28x1 크기로 변환)
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1).astype('float32')


In [ ]:
# 코드 설명
# 정규화:
# 원래 MNIST 데이터셋의 픽셀 값은 0부터 255 사이의 정수로 되어 있습니다.
# 이 값을 255로 나누면 각 픽셀이 0~1 범위의 실수 값으로 변환됩니다.
# 정규화 이유:

# 수렴 속도 향상: 머신러닝 모델, 특히 신경망 모델에서는 입력 데이터를 정규화하면 학습이 더 빨라지고 수렴 속도가 향상됩니다.
# 모델 안정성 향상: 0~1 범위로 정규화하면 큰 값이 신경망의 활성화 함수에 입력될 때 발생할 수 있는 수치적 불안정성을 줄여줍니다.
# 성능 개선: 정규화를 통해 데이터가 일정한 범위를 가지게 되면, 최적화를 위한 경사하강법이 더욱 효율적으로 작동합니다.
# 정규화 이후 데이터 범위:

# X_train과 X_test의 값은 이제 0~1 사이에 위치하게 됩니다.
# 예: 만약 어떤 픽셀 값이 255였다면, 정규화 후에는 255 / 255 = 1.0이 됩니다.

In [6]:
# 이미지 픽셀 값을 0~1 범위로 정규화
X_train /= 255
X_test /= 255


In [ ]:
# 코드 설명
# to_categorical 함수:
# to_categorical 함수는 정수형 레이블을 원-핫 인코딩 벡터로 변환합니다.
# 예를 들어, 레이블 3은 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]과 같이 10차원 벡터에서 세 번째 위치만 1이고 나머지는 0인 형태로 변환됩니다.
# 10은 클래스의 개수를 지정하는 매개변수로, 0~9의 10개의 숫자를 나타냅니다.

# 원-핫 인코딩 이유:
# 다중 클래스 분류 문제에서 신경망이 레이블을 이해할 수 있도록 정수형 레이블을 벡터 형태로 변환합니다.
# 신경망의 출력층에서는 각 클래스에 해당하는 확률을 출력하므로, 목표 레이블도 같은 형태로 맞춰주는 것이 필요합니다.

# 변환 예시:
# 원래 y_train과 y_test는 정수 레이블로 [5, 0, 4, 1, ...]와 같이 각 이미지가 나타내는 숫자(0~9)를 포함하고 있었습니다.
# 원-핫 인코딩을 적용하면 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]와 같은 형식의 벡터로 변환됩니다.

# 결과 데이터의 형태:
# y_train과 y_test는 이제 각각 (60000, 10) 및 (10000, 10)의 형태를 가지며, 각 벡터는 특정 숫자에 대한 레이블을 원-핫 인코딩으로 표현한 10차원 벡터입니다.

In [7]:
# 레이블(정답)을 원-핫 인코딩 (0~9 숫자를 10개의 클래스 값으로 변환)
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# 모델링

## CNN 모델 정의

In [ ]:
# 첫 번째 합성곱 층(Conv2D): 32개의 3x3 필터를 적용하여 이미지에서 특징을 추출합니다. 활성화 함수로 ReLU를 사용합니다.
# 최대 풀링 층(MaxPooling2D): 2x2 크기의 풀링 필터를 사용하여 차원을 줄이고, 중요한 특징만 남깁니다.
# 두 번째 합성곱 층과 최대 풀링 층도 같은 방식으로 적용되지만, 필터 개수를 64개로 설정합니다.
# Flatten 층: 합성곱과 풀링을 거친 출력을 1차원 벡터로 변환하여 완전 연결층에 전달합니다.
# 완전 연결층(Dense): 128개의 뉴런을 가지며, 활성화 함수로 ReLU를 사용합니다.
# 출력층(Dense): 10개의 클래스를 분류하기 위해 Softmax 활성화 함수를 사용합니다.

In [8]:
# CNN 모델 정의
model = Sequential()

# 첫 번째 합성곱 층 (Conv2D): 32개의 3x3 필터, 활성화 함수: ReLU, 입력 크기: 28x28x1
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))

# 풀링 층 (MaxPooling2D): 2x2 크기의 풀링 필터
model.add(MaxPooling2D(pool_size=(2, 2)))

# 두 번째 합성곱 층 (Conv2D): 64개의 3x3 필터, 활성화 함수: ReLU
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))

# 두 번째 풀링 층 (MaxPooling2D): 2x2 크기의 풀링 필터
model.add(MaxPooling2D(pool_size=(2, 2)))

# 완전 연결층에 전달하기 위해 Flatten으로 데이터를 1차원으로 변환
model.add(Flatten())

# 완전 연결층 (Dense): 128개의 노드, 활성화 함수: ReLU
model.add(Dense(128, activation='relu'))

# 출력층 (Dense): 10개의 노드, 활성화 함수: Softmax (10개의 클래스)
model.add(Dense(10, activation='softmax'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


## 모델 컴파일

In [ ]:
# 손실 함수로 categorical_crossentropy를 사용하며, 이는 다중 클래스 분류 문제에서 사용하는 손실 함수입니다.
# 최적화 방법으로 Adam을 사용하여 경사 하강법을 최적화합니다.
# 성능 평가 지표로 **accuracy(정확도)**를 사용합니다.

In [9]:
# 모델 컴파일 (손실 함수: Categorical Crossentropy, 최적화 방법: Adam, 평가지표: Accuracy)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

## 모델 학습

In [10]:
# 10 에포크 동안 학습하며, 배치 크기 128로 설정하여 학습을 진행합니다.
# 학습 중간에 테스트 데이터를 사용해 검증도 수행합니다.

In [15]:
# 모델 학습
history = model.fit(X_train, y_train, epochs=10, batch_size=128, validation_data=(X_test, y_test))

Epoch 1/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.8608 - loss: 0.4732 - val_accuracy: 0.9786 - val_loss: 0.0642
Epoch 2/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9807 - loss: 0.0624 - val_accuracy: 0.9854 - val_loss: 0.0448
Epoch 3/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9879 - loss: 0.0403 - val_accuracy: 0.9884 - val_loss: 0.0371
Epoch 4/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9904 - loss: 0.0295 - val_accuracy: 0.9903 - val_loss: 0.0325
Epoch 5/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9928 - loss: 0.0227 - val_accuracy: 0.9890 - val_loss: 0.0322
Epoch 6/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9949 - loss: 0.0165 - val_accuracy: 0.9913 - val_loss: 0.0302
Epoch 7/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9956 - loss: 0.0147 - val_accuracy: 0.9902 - val_loss: 0.0308
Epoch 8/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9963 - loss: 0.0110 - val_accuracy: 0.

## 모델 평가

In [ ]:
# 학습이 완료되면 테스트 데이터셋에서 모델 성능을 평가하고 정확도(accuracy)를 출력합니다.

In [11]:
# model.evaluate(X_test, y_test):

# model.evaluate 함수는 입력된 **테스트 데이터셋(X_test, y_test)**을 이용해 학습된 모델의 성능을 평가합니다.
# 이 함수는 모델이 X_test 데이터를 입력받아 예측한 값과 실제 라벨(y_test) 간의 **손실(loss)**과 **정확도(accuracy)**를 계산하여 반환합니다.
# loss: 테스트 데이터에 대한 모델의 평균 손실 값입니다.
# accuracy: 테스트 데이터에서 예측이 정답과 일치한 비율로, 모델의 분류 정확도를 나타냅니다.
# 출력 형식:

# print(f'Test Accuracy: {accuracy * 100:.2f}%')는 정확도를 % 형태로 보기 좋게 출력합니다.
# accuracy * 100은 정확도를 퍼센트로 변환하며, :.2f는 소수점 둘째 자리까지 표시하도록 형식을 지정합니다.


In [14]:
# 모델 평가
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy * 100:.2f}%')


313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0974 - loss: 2.3138
Test Accuracy: 9.91%


In [ ]:
# 평가 결과 해석:

# loss는 모델의 손실을 나타내며, 값이 낮을수록 테스트 데이터에서 잘 작동하고 있음을 의미합니다.
# accuracy는 테스트 데이터의 예측 정확도를 나타내며, 값이 높을수록 모델이 테스트 데이터에 대해 정확한 예측을 수행했음을 의미합니다.
# 예: Test Accuracy: 99.05%라면, 모델이 테스트 데이터에서 99.05%의 정확도를 기록한 것입니다.

## 예측

In [ ]:
# 학습된 모델을 사용해 테스트 데이터에 대한 예측값을 생성합니다.

# 결과 출력
# 모델 학습이 완료되면 다음과 같은 정보가 출력됩니다:

# Test Accuracy: 테스트 데이터셋에서의 정확도를 퍼센트로 표시합니다

In [13]:
# 예측 (테스트 데이터에 대한 예측)
predictions = model.predict(X_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


# 종합

In [17]:
# 필요한 라이브러리 임포트
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# MNIST 데이터셋 로드 (훈련 데이터와 테스트 데이터 분리)
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# 데이터 전처리 (28x28 크기의 이미지를 28x28x1 크기로 변환)
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1).astype('float32')

# 이미지 픽셀 값을 0~1 범위로 정규화
X_train /= 255
X_test /= 255

# 레이블(정답)을 원-핫 인코딩 (0~9 숫자를 10개의 클래스 값으로 변환)
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# CNN 모델 정의
model = Sequential()

# 첫 번째 합성곱 층 (Conv2D): 32개의 3x3 필터, 활성화 함수: ReLU, 입력 크기: 28x28x1
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))

# 풀링 층 (MaxPooling2D): 2x2 크기의 풀링 필터
model.add(MaxPooling2D(pool_size=(2, 2)))

# 두 번째 합성곱 층 (Conv2D): 64개의 3x3 필터, 활성화 함수: ReLU
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))

# 두 번째 풀링 층 (MaxPooling2D): 2x2 크기의 풀링 필터
model.add(MaxPooling2D(pool_size=(2, 2)))

# 완전 연결층에 전달하기 위해 Flatten으로 데이터를 1차원으로 변환
model.add(Flatten())

# 완전 연결층 (Dense): 128개의 노드, 활성화 함수: ReLU
model.add(Dense(128, activation='relu'))

# 출력층 (Dense): 10개의 노드, 활성화 함수: Softmax (10개의 클래스)
model.add(Dense(10, activation='softmax'))

# 모델 컴파일 (손실 함수: Categorical Crossentropy, 최적화 방법: Adam, 평가지표: Accuracy)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델 학습
history = model.fit(X_train, y_train, epochs=10, batch_size=128, validation_data=(X_test, y_test))

# 모델 평가
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

# 예측 (테스트 데이터에 대한 예측)
predictions = model.predict(X_test)


Epoch 1/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.8497 - loss: 0.5050 - val_accuracy: 0.9795 - val_loss: 0.0639
Epoch 2/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9807 - loss: 0.0628 - val_accuracy: 0.9851 - val_loss: 0.0453
Epoch 3/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9875 - loss: 0.0414 - val_accuracy: 0.9812 - val_loss: 0.0568
Epoch 4/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9900 - loss: 0.0324 - val_accuracy: 0.9887 - val_loss: 0.0336
Epoch 5/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9925 - loss: 0.0253 - val_accuracy: 0.9903 - val_loss: 0.0306
Epoch 6/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9944 - loss: 0.0192 - val_accuracy: 0.9897 - val_loss: 0.0333
Epoch 7/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9947 - loss: 0.0160 - val_accuracy: 0.9913 - val_loss: 0.0272
Epoch 8/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9966 - loss: 0.0117 - val_accuracy: 0.